In [9]:
import requests
import pandas as pd

# User Inputs
COINS = ["BTC", "BANANAS31"]
START_DATE = "2025-03-18"
END_DATE = "2025-03-19"
INTERVAL = 5                        # Minutes (1, 5, 60, etc.)

# Convert dates to timestamps
start_ts = int(pd.to_datetime(START_DATE).timestamp())
end_ts = int(pd.to_datetime(END_DATE).timestamp())

# Fetch data
url = "https://min-api.cryptocompare.com/data/v2/histominute"
data_dict = {}

for coin in COINS:
    response = requests.get(url, params={
        "fsym": coin,
        "tsym": "USD",
        "aggregate": INTERVAL,
        "toTs": end_ts,
        "limit": 2000  # Max per request
    })

    if response.status_code == 200 and response.json()["Response"] == "Success":
        df = pd.DataFrame(response.json()["Data"]["Data"])
        if df.empty:
            print(f"No data for {coin}")
            continue
        df = df[["time", "close"]].assign(
            time=pd.to_datetime(df["time"], unit="s"),
            close=df["close"].astype(float)
        )
        df = df[(df["time"] >= START_DATE) & (df["time"] <= END_DATE)]
        data_dict[coin] = df.set_index("time")["close"].rename(f"{coin}_close")
    else:
        print(f"Failed for {coin}: Check coin or connection")

# Save to Excel
if data_dict:
    df = pd.concat(data_dict.values(), axis=1, join="inner").reset_index()
    df.columns = ["time"] + [f"{coin}_close" for coin in COINS]
    print("Preview:")
    print(df.head())
    df.to_excel("cryptocompare_coins.xlsx", index=False, sheet_name="Coin_Data")
    print("Saved to cryptocompare_coins.xlsx")
else:
    print("No data fetched")

Failed for BTC: Check coin or connection
Failed for BANANAS31: Check coin or connection
No data fetched
